In [50]:
library('tidyverse'); packageVersion('tidyverse')
library("phyloseq"); packageVersion('phyloseq')
library("ggpubr"); packageVersion('ggpubr')
library("vegan"); packageVersion('vegan')
library("MASS"); packageVersion('MASS')
library("scales"); packageVersion('scales')
library("picante"); packageVersion('picante')
library("caret"); packageVersion('caret')
library("AppliedPredictiveModeling"); packageVersion('AppliedPredictiveModeling')
library("ranger"); packageVersion('ranger')
library("e1071"); packageVersion('e1071')
library("randomForest"); packageVersion('randomForest')
library("alluvial"); packageVersion('alluvial')

[1] ‘1.3.1’

[1] ‘1.36.0’

[1] ‘0.4.0’

[1] ‘2.5.7’

[1] ‘7.3.54’

...

[1] ‘1.1.7’

[1] ‘0.13.1’

[1] ‘1.7.9’

[1] ‘4.6.14’

[1] ‘0.1.2’

In [51]:
# Theme set and Color Palettes
theme_set(theme_pubr())
rootstock_palette <- c('#1b9e77', '#f0a4af', '#7570b3')
scion_palette <- c('#ed254e', '#0e79b2')
site_palette <- c('#e6ab02', '#281c39', '#12664c')
compartment_palette <- c("#5a1991", "#139d08", "#5c3c0d") #https://lospec.com/palette-list/famicube
safe_colorblind_palette <- c("#88CCEE", "#CC6677", "#DDCC77", "#AA4499", "#332288", "#117733", 
                             "#661100", "#999933", "#44AA99", "#882255", "#6699CC", "#888888")
# Set seed for analysis
set.seed(1154829343)

In [52]:
# Functions
# Function to return metadata df from phyloseq object
pssd2veg <- function(physeq) {
  # From a phyloseq object return a dataframe of the sample metadata for use in vegan
  # From: https://jacobrprice.github.io/2017/08/26/phyloseq-to-vegan-and-back.html
  sd <- sample_data(physeq)
  return(as(sd,"data.frame"))
}
# Function to plot or run a linear model on a ASV from a phyloseq object
plot_deseq2_DiffAbunMicob <- function(physeq_obj, ASV_number){
  temp_sample_tab <- pssd2veg(physeq_obj)
  otu_matrix <- as(otu_table(physeq_obj), "matrix")
  TEMP <- data.frame(ASV_count = otu_matrix[ASV_number,])
  TEMP2<- cbind(temp_sample_tab, TEMP)
  #Anova(lm(ASV_count ~ Compartment*Rootstock + Compartment*Irrigation + Rootstock*Irrigation + Block, data = TEMP2), type = "III")
  ggplot(TEMP2, aes(plant_body_site, ASV_count, fill= rootstock)) + geom_boxplot(outlier.shape = NA) + geom_point(position = position_jitterdodge(jitter.width = 0.2)) + scale_fill_manual(name = "Rootstock", values=rootstock_palette) + scale_y_continuous(name="Abundance") + xlab("Compartment") + theme(legend.position="right", axis.title = element_text(size = 14), axis.text = element_text(size = 12), plot.title = element_text(size=22))
}

# Function to plot confusion matrix using ggtile plot from a confussion matrix object
# By user: Enrique Perez Herrero 
# on https://stackoverflow.com/questions/46063234/how-to-produce-a-confusion-matrix-and-find-the-misclassification-rate-of-the-na%C3%AF
ggplotConfusionMatrix <- function(m){
  mytitle <- paste("Accuracy", percent_format()(m$overall[1]),
                   "Kappa", percent_format()(m$overall[2]))
  
  d <- as.data.frame.matrix(m$table)
  drn <- colnames(d)
  drr <- rownames(d)
  drs <- rowSums(d)
  d <- d %>% mutate_if(is.numeric, funs(./drs))
  d <- d %>% gather(x, value)
  Y <- cbind(as.data.frame(m$table), Proportion = d$value)
  Y$Reference <- fct_rev(Y$Reference) # Added this line to get a downward diagonal 
  p <-
    ggplot(data = Y, aes(x = Reference, y = Prediction, fill= Proportion)) +
    geom_tile( colour = "white") +
    scale_fill_gradient(low = "white", high = "#14A02E", na.value = "white", limits=c(0,1)) +
    ggtitle(mytitle) +
    theme(legend.position = "right", axis.text.x = element_text(angle = 60, hjust = 1)) +
    guides(fill = guide_colorbar(frame.colour = "black", ticks = FALSE))
  return(p)
}

In [60]:
# HERE THE FUNCTION FOR MLing

# In case I want interactions
# Example below
# Compartment_Rootstock = paste(ASV_metadata.df$plant_body_site, ASV_metadata.df$rootstock, sep = "_"),

MachineLearning_RF_ranger <- function(PHYSEQ_OBJ_1, GROUPING, TREES) {
    # Remove ASV Table and meta data from phyloseq objects
    ASV.df <- as.data.frame(otu_table(PHYSEQ_OBJ_1))
    ASV_metadata.df <- as.data.frame(sample_data(PHYSEQ_OBJ_1))
    # Format ASV table to be used for machine learning applications and make metadata df
    ASV.df <- t(ASV.df)
    ASV_meta.df <- data.frame(Sample = rownames(ASV_metadata.df), Year = ASV_metadata.df$year, Scion = ASV_metadata.df$scion, 
                              Rootstock = ASV_metadata.df$rootstock, Compartment = ASV_metadata.df$plant_body_site, 
                              Site = ASV_metadata.df$site, Rootstock_by_Scion = ASV_metadata.df$rootstock_by_scion, 
                              Rootstock_by_Site = ASV_metadata.df$rootstock_by_site, 
                              Rootstock_by_Compartment = ASV_metadata.df$rootstock_by_compartment, 
                              Rootstock_by_Year = ASV_metadata.df$rootstock_by_year, 
                              Scion_by_Site = ASV_metadata.df$scion_by_site, 
                              Scion_by_Compartment = ASV_metadata.df$scion_by_compartment, 
                              Scion_by_Year = ASV_metadata.df$scion_by_year, 
                              Year_by_Compartment = ASV_metadata.df$year_by_compartment, 
                              Year_by_Site = ASV_metadata.df$year_by_site, 
                              Compartment_by_Site = ASV_metadata.df$compartment_by_site,
                              Sugar_content = ASV_metadata.df$brix_2_breaks)
    ASV_prefiltered.df <- cbind(ASV.df, ASV_meta.df)
    # ~80:20 split train/test datasets while respecting groups (i.e. sampling the same number of samples from each label)
    # 594 total samples * 0.8 = 474
    # 474/Factor level = sample_n(#)
    # e.g. for compartment 474/3 = 158
    ### DUE TO THE DESIGN FOR SOME OF THE FACTORS BEING IMBALANCED I AM JUST GOING TO TAKE A RANDOM 80/20 SPLIT. ####
    #if ((GROUPING) == "Compartment"){
    #  train_index <- as.data.frame(ASV_prefiltered.df %>% group_by_(GROUPING) %>% sample_n(158))
    #} else if ((GROUPING) == "Site"){
    #  train_index <- as.data.frame(ASV_prefiltered.df %>% group_by_(GROUPING) %>% sample_n(158))
    #} else if ((GROUPING) == "Scion"){
    #  train_index <- as.data.frame(ASV_prefiltered.df %>% group_by_(GROUPING) %>% sample_n(237))
    #} else if ((GROUPING) == "Year"){
    #  train_index <- as.data.frame(ASV_prefiltered.df %>% group_by_(GROUPING) %>% sample_n(237))
    #} else if ((GROUPING) == "Rootstock"){
    #  train_index <- as.data.frame(ASV_prefiltered.df %>% group_by_(GROUPING) %>% sample_n(158))
    #}
    train_index <- as.data.frame(ASV_prefiltered.df %>% sample_n(475))
    rownames(train_index) <- train_index$Sample
    train_index <- match(rownames(train_index), rownames(ASV_prefiltered.df))
    train_x <- as.data.frame(ASV.df[train_index, ])
    test_y <- as.data.frame(ASV.df[-train_index, ])
    # Train set, 474
    train_x$Sample <- rownames(train_x)
    Training_meta.df <- merge(train_x, ASV_meta.df, by = 'Sample')
    train_x <- subset(Training_meta.df, select = -c(Compartment, Site, Scion, Year, Rootstock, Rootstock_by_Scion, 
                                                    Rootstock_by_Site, Rootstock_by_Compartment, Rootstock_by_Year, 
                                                    Scion_by_Site, Scion_by_Compartment, Scion_by_Year, Year_by_Compartment, 
                                                    Year_by_Site, Compartment_by_Site, Sugar_content))
    rownames(train_x) <- train_x$Sample
    train_x <- subset(train_x, select = -c(Sample))
    Training_meta.df <- subset(Training_meta.df, select = c(Compartment, Site, Scion, Year, Rootstock, Rootstock_by_Scion, 
                                                    Rootstock_by_Site, Rootstock_by_Compartment, Rootstock_by_Year, 
                                                    Scion_by_Site, Scion_by_Compartment, Scion_by_Year, Year_by_Compartment, 
                                                    Year_by_Site, Compartment_by_Site, Sugar_content))
    rownames(Training_meta.df) <- Training_meta.df$Sample 
    # Test set, 120 samples
    test_y$Sample <- rownames(test_y)
    Testing_meta.df <- merge(test_y, ASV_meta.df, by = "Sample")
    test_y <- subset(Testing_meta.df, select = -c(Compartment, Site, Scion, Year, Rootstock, Rootstock_by_Scion, 
                                                    Rootstock_by_Site, Rootstock_by_Compartment, Rootstock_by_Year, 
                                                    Scion_by_Site, Scion_by_Compartment, Scion_by_Year, Year_by_Compartment, 
                                                    Year_by_Site, Compartment_by_Site, Sugar_content))
    rownames(test_y) <- test_y$Sample
    test_y <- subset(test_y, select = -c(Sample))
    Testing_meta.df <- subset(Testing_meta.df, select = c(Compartment, Site, Scion, Year, Rootstock, Rootstock_by_Scion, 
                                                    Rootstock_by_Site, Rootstock_by_Compartment, Rootstock_by_Year, 
                                                    Scion_by_Site, Scion_by_Compartment, Scion_by_Year, Year_by_Compartment, 
                                                    Year_by_Site, Compartment_by_Site, Sugar_content))
    rownames(Testing_meta.df) <- Testing_meta.df$Sample 
    # Training model
    Training_grid <- expand.grid(.mtry = seq(10, length(train_x), round(length(train_x)*0.1)), .splitrule= "gini",
                                 .min.node.size = c(1, 5, 10))
    train_control <- trainControl(method="cv", number=10)
    RF_CM <- list()
    RF_CM[["RF_model"]] <- train(x = train_x, y = Training_meta.df[[GROUPING]], method = "ranger", importance = "impurity",
                                 tuneGrid = Training_grid, trControl = train_control, num.trees = TREES)
    RF_prediction_3 <- predict(RF_CM[["RF_model"]], test_y)
    RF_CM[["CMatrix"]] <- confusionMatrix(RF_prediction_3, as.factor(Testing_meta.df[[GROUPING]]), mode = "everything")
    RF_CM[["CMatrixPLOT"]] <- ggplotConfusionMatrix(RF_CM[["CMatrix"]])
    RF_CM[["VarImporance"]] <- varImp(RF_CM[["RF_model"]])
    return(RF_CM)
}

In [61]:
phy_vst <- readRDS("phyloseq_16s_no_soil_filtered_vst_dataset.rds")
#### 2 Breaks Brix ####
phy_vst@sam_data$brix_2_breaks <- cut(phy_vst@sam_data$brix, breaks = c(-Inf, 7, Inf), labels = c("Pre-ripening", "Ripening")) # Check if I want to use these terms
summary(sample_data(phy_vst))

     barcode.sequence            LinkerPrimerSequence extraction_num
 AACACATGGGTT:  1     GTGTGYCAGCMGCCGCGGTAA:594       10     :  1   
 AACAGGTCTCTG:  1                                     100    :  1   
 AACATTGCAGGT:  1                                     101    :  1   
 AACGAATACCAC:  1                                     103    :  1   
 AACGACACGCTT:  1                                     104    :  1   
 AACGCGAAATTC:  1                                     105    :  1   
 (Other)     :588                                     (Other):588   
  date_collect          extract_date col_week           block    
 Min.   :2018-06-19   11212019: 24   1:115    089 OLCESE 24: 44  
 1st Qu.:2018-07-10   1302020 : 24   2:117    005 SEC D    : 43  
 Median :2018-08-01   7022020 : 24   4:123    103 SEC B    : 43  
 Mean   :2018-12-04   7072020 : 24   6:122    018 CARPENTER: 42  
 3rd Qu.:2019-06-27   7092020 : 24   7:117    RIP 760      : 42  
 Max.   :2019-07-25   7302020 : 24            (Other

In [64]:
#RF_CM_SITE <- MachineLearning_RF_ranger(phy_vst, "Site", 237)
#saveRDS(RF_CM_SITE, "RF_CM_CV10_237tree_Site.rand_split.rds")
#RF_CM_ROOTSTOCK <- MachineLearning_RF_ranger(phy_vst, "Rootstock", 255)
#saveRDS(RF_CM_ROOTSTOCK, "RF_CM_CV10_255tree_Rootstock.rand_split.rds")
#RF_CM_COMPARTMENT <- MachineLearning_RF_ranger(phy_vst, "Compartment", 63)
#saveRDS(RF_CM_COMPARTMENT, "RF_CM_CV10_63tree_Compartment.rand_split.rds")
#RF_CM_YEAR <- MachineLearning_RF_ranger(phy_vst, "Year", 477)
#saveRDS(RF_CM_YEAR, "RF_CM_CV10_477tree_Year.rand_split.rds")
#RF_CM_SCION <- MachineLearning_RF_ranger(phy_vst, "Scion", 433)
#saveRDS(RF_CM_SCION, "RF_CM_CV10_433tree_Scion.rand_split.rds")

RF_CM_Sugar <- MachineLearning_RF_ranger(phy_vst, "Sugar_content", 267)
saveRDS(RF_CM_Sugar, "RF_CM_CV10_267tree_Sugar.rand_split.rds")

In [65]:
saveRDS(RF_CM_Sugar, "RF_CM_CV10_267tree_Sugar.rand_split.rds")
RF_CM_Sugar

$RF_model
Random Forest 

 475 samples
7981 predictors
   2 classes: 'Pre-ripening', 'Ripening' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 427, 428, 427, 427, 427, 428, ... 
Resampling results across tuning parameters:

  mtry  min.node.size  Accuracy   Kappa     
    10   1             0.6064273  0.06634389
    10   5             0.6148050  0.08626614
    10  10             0.6001773  0.05455594
   808   1             0.6801862  0.30550459
   808   5             0.6781028  0.30797065
   808  10             0.6824025  0.31636428
  1606   1             0.6654255  0.27556381
  1606   5             0.6803191  0.31102413
  1606  10             0.6824025  0.31577705
  2404   1             0.6866578  0.32805246
  2404   5             0.6739805  0.29584841
  2404  10             0.6781028  0.30973742
  3202   1             0.6781028  0.30639935
  3202   5             0.6845745  0.32068700
  3202  10             0.6866135  0.32678974
  4000   1         

ERROR: Error in value[[3L]](cond): could not open file '/tmp/RtmppEXoWr/file2094fe4c8e7.png'


plot without title

In [63]:
# Joint predictions (2-ways)

# Add new metadata columns to accommodate joint models
sample_data(phy_vst)$rootstock_by_scion <- as.factor(paste(sample_data(phy_vst)$rootstock, sample_data(phy_vst)$scion, sep="_"))
sample_data(phy_vst)$rootstock_by_site <- as.factor(paste(sample_data(phy_vst)$rootstock, sample_data(phy_vst)$site, sep="_"))
sample_data(phy_vst)$rootstock_by_compartment <- as.factor(paste(sample_data(phy_vst)$rootstock, sample_data(phy_vst)$plant_body_site, sep="_"))
sample_data(phy_vst)$rootstock_by_year <- as.factor(paste(sample_data(phy_vst)$rootstock, sample_data(phy_vst)$year, sep="_"))

sample_data(phy_vst)$scion_by_site <- as.factor(paste(sample_data(phy_vst)$scion, sample_data(phy_vst)$site, sep="_"))
sample_data(phy_vst)$scion_by_compartment <- as.factor(paste(sample_data(phy_vst)$scion, sample_data(phy_vst)$plant_body_site, sep="_"))
sample_data(phy_vst)$scion_by_year <- as.factor(paste(sample_data(phy_vst)$scion, sample_data(phy_vst)$year, sep="_"))

sample_data(phy_vst)$year_by_compartment <- as.factor(paste(sample_data(phy_vst)$year, sample_data(phy_vst)$plant_body_site, sep="_"))
sample_data(phy_vst)$year_by_site <- as.factor(paste(sample_data(phy_vst)$year, sample_data(phy_vst)$site, sep="_"))

sample_data(phy_vst)$compartment_by_site <- as.factor(paste(sample_data(phy_vst)$plant_body_site, sample_data(phy_vst)$site, sep="_"))

summary(sample_data(phy_vst))

     barcode.sequence            LinkerPrimerSequence extraction_num
 AACACATGGGTT:  1     GTGTGYCAGCMGCCGCGGTAA:594       10     :  1   
 AACAGGTCTCTG:  1                                     100    :  1   
 AACATTGCAGGT:  1                                     101    :  1   
 AACGAATACCAC:  1                                     103    :  1   
 AACGACACGCTT:  1                                     104    :  1   
 AACGCGAAATTC:  1                                     105    :  1   
 (Other)     :588                                     (Other):588   
  date_collect          extract_date col_week           block    
 Min.   :2018-06-19   11212019: 24   1:115    089 OLCESE 24: 44  
 1st Qu.:2018-07-10   1302020 : 24   2:117    005 SEC D    : 43  
 Median :2018-08-01   7022020 : 24   4:123    103 SEC B    : 43  
 Mean   :2018-12-04   7072020 : 24   6:122    018 CARPENTER: 42  
 3rd Qu.:2019-06-27   7092020 : 24   7:117    RIP 760      : 42  
 Max.   :2019-07-25   7302020 : 24            (Other

In [29]:
##### Join Predictions

#RF_CM_RxS <- MachineLearning_RF_ranger(phy_vst, "Rootstock_by_Scion", 400)
#saveRDS(RF_CM_RxS, "RF_CM_CV10_400tree_Rootstock_by_Scion.rand_split.rds")

RF_CM_RxSi <- MachineLearning_RF_ranger(phy_vst, "Rootstock_by_Site", 400)
saveRDS(RF_CM_RxSi, "RF_CM_CV10_400tree_Rootstock_by_Site.rand_split.rds")

RF_CM_RxC <- MachineLearning_RF_ranger(phy_vst, "Rootstock_by_Compartment", 400)
saveRDS(RF_CM_RxC, "RF_CM_CV10_400tree_Rootstock_by_Compartment.rand_split.rds")

RF_CM_RxY <- MachineLearning_RF_ranger(phy_vst, "Rootstock_by_Year", 400)
saveRDS(RF_CM_RxY, "RF_CM_CV10_400tree_Rootstock_by_Year.rand_split.rds")



RF_CM_SxSi <- MachineLearning_RF_ranger(phy_vst, "Scion_by_Site", 400)
saveRDS(RF_CM_SxSi, "RF_CM_CV10_400tree_Scion_by_Site.rand_split.rds")

RF_CM_SxC <- MachineLearning_RF_ranger(phy_vst, "Scion_by_Compartment", 400)
saveRDS(RF_CM_SxC, "RF_CM_CV10_400tree_Scion_by_Compartment.rand_split.rds")

RF_CM_SxY <- MachineLearning_RF_ranger(phy_vst, "Scion_by_Year", 400)
saveRDS(RF_CM_SxY, "RF_CM_CV10_400tree_Scion_by_Year.rand_split.rds")



RF_CM_YxC <- MachineLearning_RF_ranger(phy_vst, "Year_by_Compartment", 400)
saveRDS(RF_CM_YxC, "RF_CM_CV10_400tree_Year_by_Compartment.rand_split.rds")

RF_CM_YxSi <- MachineLearning_RF_ranger(phy_vst, "Year_by_Site", 400)
saveRDS(RF_CM_YxSi, "RF_CM_CV10_400tree_Year_by_Site.rand_split.rds")



RF_CM_CxSi <- MachineLearning_RF_ranger(phy_vst, "Compartment_by_Site", 400)
saveRDS(RF_CM_CxSi, "RF_CM_CV10_400tree_Compartment_by_Site.rand_split.rds")